<table width = "100%">
  <tr style="background-color:white;">
    <!-- QWorld Logo -->
    <td style="text-align:left;width:200px;"> 
        <a href="https://qworld.net/" target="_blank"><img src="../images/QWorld.png"> </a></td>
    <td style="text-align:right;vertical-align:bottom;font-size:16px;"> 
        Prepared by <a href="https://gitlab.com/sabahuddin.ahmad" target="_blank"> Sabah Ud Din Ahmad </a></td>
    </tr> 
 </table>
 
<hr>

## Examples for QUBO Formulation

In the previous section, we learnt about the penalty method for constrained problems. Now, lets apply it to some important constrained optimization problems. 

### Travelling Salesman Problem (TSP)

Recall, given a set of cities and corresponding distances between each pair of cities, the problem is to find the shortest possible route such that a salesman visits every city exactly once and returns to the starting point / hometown. So, eventually, the salesman maximizes his sales by minimizng the total cost of travelling between the nodes. 

#### Hamiltonian Cycle

A path through a graph that visits each node (city) exactly once is called a Hamiltonian path or a **Hamiltonian cycle**. To define the mathematical formulation, the problem asks for the shortest Hamiltonian cycle that can be taken through each of the nodes. So, we will find the shortest Hamiltonian cycle in a graph $G = (V,E)$ with N nodes and costs $w_{ij}$ (cost of travelling from node $i$ to node $j$). Here $V =\{1,...N\}$ is the set of nodes and E is the edge set. 

The cycle is defined using $N^2$ binary variables $x_{i,p}$, where $i$ represents the node and $p$ represents the order in that cycle. The variable results in 1 if salesman passes node $i$ if its at position $p$ in the route.
$$x_{ip}=
\left\{
\begin{array}{ll} 
      1, & \text{node i is at position p in the route} \\
      0, & \text{otherwise} \\
\end{array}
\right.$$

#### Directed vs Undirected Graphs

The nodes are labelled 1 to N and the edge set can be:
* directed: order $(i,j)$ matters
* undirected: order doesn't matter 

We will assume our graph to be directed with edge set $(i,j)$. We can extend to undirected graphs by just considering a directed graph with $(j,i)$ added to the edge set E whenever $(i,j)$ is added to the edge set E.

#### Constraints

The problem involves the following constraints:
1. Each node can only appear once in the whole cycle.
$$\sum_{i=1}^{N} x_{i,p} = 1 \;\;\;\;\; \forall p$$

The equivalent penalty is,
$$B\sum_{p=1}^{N} \left(1-\sum_{i=1}^{N}x_{i,p}\right)^2$$

2. For each time, a node has to occur.
$$\sum_{p=1}^{N} x_{i,p} = 1 \;\;\;\;\; \forall i$$

The equivalent penalty is,
$$B\sum_{i=1}^{N} \left(1-\sum_{p=1}^{N}x_{i,p}\right)^2$$

3. If the graph isn't fully connected such that there exists $(i,j) \notin E$, so for a given route order, if $x_{i,p}$ and $x_{j,p+1}$ are both 1 (the nodes are in a given order but there is no edge linking them), there will be an energy penalty.
$$x_{i,p} + x_{j,p+1} \leq 1 \;\;\;\;\;  p=1,...,K $$

The  equivalent penalty is,
$$B\sum_{(i,j) \notin E} \sum_{p=1}^{N} x_{i,p} x_{j,p+1} > 0$$

#### Objective Function

The cost that needs to be minimized is:
$$C(x) = \sum_{(i,j) \in E} w_{ij} \sum_{p=1}^{N} x_{i,p} x_{j,p+1}$$

Now, including the constraints to get a single objective function:
$$C(x) = A\sum_{(i,j) \in E} w_{ij} \sum_{p=1}^{N} x_{i,p} x_{j,p+1} + B\sum_{p=1}^{N} \left(1-\sum_{i=1}^{N}x_{i,p}\right)^2 + B\sum_{i=1}^{N} \left(1-\sum_{p=1}^{N}x_{i,p}\right)^2 + B\sum_{(i,j) \notin E} \sum_{p=1}^{N} x_{i,p} x_{j,p+1}$$

Here, A should be small enough so not to violate the constraints whereas B is a positive penalty parameter that should be sufficiently large for the constraints to be considered. One possibility can be $0<A\max (w_{ij})<B$ (assuming that $w_{ij} \geq 0$ for each $(i,j) \in E$).

<a id="example"></a>
### Example: TSP with 3 Nodes

Let's assume we have the following fully-connected, undirected graph of 3 nodes / cities and 3 edges, $G = (3,3)$. 

<img src="../images/tsp_3.png" width="250">

We will now **determine the matrix Q** for this graph. 

Since the graph is fully connected, we must drop the last term corresponding to the 3rd constraint. Let $A=1$ and $B=20$, under the condition $0<A\max (w_{ij})<B$ since $\max (w_{ij})=15$.
So, with these simplifications,

$$C(x) = \sum_{(i,j) \in E} w_{ij} \sum_{p=1}^{3} x_{i,p} x_{j,p+1} + B\sum_{p=1}^{3} \left(1-\sum_{i=1}^{3}x_{i,p}\right)^2 + B\sum_{i=1}^{3} \left(1-\sum_{p=1}^{3}x_{i,p}\right)^2$$
$$C(x) = \sum_{(i,j) \in E} w_{ij} (x_{i1}x_{j2} + x_{i2}x_{j3} + x_{i3}x_{j4}) + B\sum_{p=1}^{3} \left(1 - (x_{1p}+x_{2p}+x_{3p})\right)^2 + B\sum_{i=1}^{3} \left(1-(x_{i1}+x_{i2}+x_{i3})\right)^2$$

Let's look at the first term.
$$\sum_{(i,j) \in E} w_{ij} (x_{i1}x_{j2} + x_{i2}x_{j3} + x_{i3}x_{j4})$$

Here, $E=\{(1,2),(1,3),(2,1),(2,3),(3,1),(3,2)\}$. You may notice that the edges are repeated as the graph is undirected. For each $(i,j) \in E$, corresponding $w_{ij}$ is multiplied with a sum of crossterms ($x_{i,p}x_{j,p+1}$) which are binary variables. The indices i and j in $w_{ij}$ denote the relevant indices of the crossterms. E.g. for $(1,2)$,
$$w_{12} (x_{11}x_{22} + x_{12}x_{23} + x_{13}x_{24})$$ where $w_{12}=15$.

Within the summation, we have a total of 6 terms, one for each $(i,j)\in E$. Summing up for all,
$$w_{12} (x_{11}x_{22} + x_{12}x_{23} + x_{13}x_{24})+..........+w_{32} (x_{31}x_{22} + x_{32}x_{23} + x_{33}x_{24})$$

where $w_{12}=w_{21}=15, w_{13}=w_{31}=14, w_{23}=w_{32}=8$.

Let's look at the second term.
$$B\sum_{p=1}^{3} \left(1 - (x_{1p}+x_{2p}+x_{3p})\right)^2$$
Expanding the sum,
$$B\left[\left(1 - (x_{11}+x_{21}+x_{31})\right)^2+\left(1 - (x_{12}+x_{22}+x_{32})\right)^2+\left(1 - (x_{13}+x_{23}+x_{33})\right)^2\right]$$
This can be simplified by expanding the quadratic expressions to get linear terms ($x_{ip}$), quadratic terms ($x_{ip}^2$), and a constant 3B.

Let's look at the third term.
$$B\sum_{i=1}^{3} \left(1-(x_{i1}+x_{i2}+x_{i3})\right)^2$$
Expanding the sum,
$$B\left[\left(1 - (x_{11}+x_{12}+x_{13})\right)^2+\left(1 - (x_{21}+x_{22}+x_{23})\right)^2+\left(1 - (x_{31}+x_{32}+x_{33})\right)^2\right]$$
This can be simplified by expanding the quadratic expressions to get linear terms ($x_{ip}$), quadratic terms ($x_{ip}^2$), and a constant 3B.

Combining all these, 

$$C(x) = w_{12} (x_{11}x_{22} + x_{12}x_{23} + x_{13}x_{24})+..........+w_{32} (x_{31}x_{22} + x_{32}x_{23} + x_{33}x_{24})+ B\left[\left(1 - (x_{11}+x_{21}+x_{31})\right)^2+\left(1 - (x_{12}+x_{22}+x_{32})\right)^2+\left(1 - (x_{13}+x_{23}+x_{33})\right)^2\right]+ B\left[\left(1 - (x_{11}+x_{12}+x_{13})\right)^2+\left(1 - (x_{21}+x_{22}+x_{23})\right)^2+\left(1 - (x_{31}+x_{32}+x_{33})\right)^2\right]$$

For ease in calculations, we can replace double index binary variables with single index binary variables.

$$(x_{11}, x_{12}, x_{13}, x_{14}, x_{21}, x_{22}, x_{23}, x_{24}, x_{31}, x_{32}, x_{33}, x_{34}) = (x_1, x_2, x_3, x_4, x_5, x_6, x_7,x_8,x_9,x_{10},x_{11}, x_{12})$$

To get the matrix Q, we can replace any term $x_i$ with $x_i^2$ and vice versa since these are binary variables (this doesnt apply to products $x_i x_j$). We will ignore the constant $6B$ while constructing the matrix.

So, with these modifications, our simplified objective function expression takes the desired form:

$$\min_{x \in {0,1}^n} x^T Q x$$

where:

$$x^T = \begin{pmatrix}
x_1 & x_2 & x_3 & x_4 & x_5 & x_6 & x_7 & x_8 & x_9 & x_{10} & x_{11} & x_{12}
\end{pmatrix}$$

and we obtain a $12\times 12$ upper diagonal matrix Q. 

### Task 1

Let's assume we have the following fully-connected, undirected graph of 4 nodes / cities and 6 edges, $G = (4,6)$. 

<img src="../images/tsp_1.png" width="250">

Determine the matrix Q for this graph. (You may assume suitable values for A and B). 

[Click here for solution](QUBO_Examples_TSP_Solutions.ipynb#task1)

***

### Fixing Initial and Final Node

In order to simplify the algebraic expressions for a particular graph, we can have a specific case where the initial and final node is fixed. This reduces the number of variables. 

Let's continue with [Example: TSP with 3 Nodes](#example).

### Task 2

Assume Node 1 is the first and the final node in our cycle.

$$x_{11}=1 \;\;\;\;\; x_{14}=1$$
$$x_{12}=x_{13}=x_{21}=x_{24}=x_{31}=x_{34}=0$$

With these values of the variables, simplify the objective function and obtain the corresponding matrix Q.  

[Click here for solution](QUBO_Examples_TSP_Solutions.ipynb#task2)

***

### Task 3

Input matrix Q calculated in Task 2 to the function *qubo_solver()* and determine $x$ which minimizes $x^T Qx$ and the corresponding minimum value.

In [ ]:
# Access the qubo_solver() function
%run qubo_functions.py

# Define the Q matrix



# Pass the matrix as an argument to the function
qubo_solver(Q)

[Click here for solution](QUBO_Examples_TSP_Solutions.ipynb#task3)

***

### Task 4

Using the QUBO algebraic expression and testing all possibilities of $x$, verify your result for Task 3. 

(You may use a Python code for this task).

In [ ]:
#Create a function to evaluate the value of objective function for each x.

def tsp_task_4(x):
    #INSERT YOUR CODE HERE!
    
    
    
    return y

#Minimize the function for all possibilites of x.
#The following code generates the possile permutations of x and calculates the value of the objectve funtion for each.
import numpy as np
import itertools
possible_values = {}
vec_permutations = itertools.product([0,1], repeat=4)    # A list of all the possible permutations for x vector
for permutation in vec_permutations:
    x = np.array([[var] for var in permutation])         # Converts the permutation into a column vector
    value = tsp_task_4(x)
    possible_values[value[0]] = x
    vector = tuple(x.T[0])
    #print("Vector x =", vector, "; Value =",int(value))  # Displays every vector with its corresponding value
    
min_value = min(possible_values.keys())                  # Lowest value of the objective function
opt_vector = tuple(possible_values[min_value].T[0])      # Optimum x corresponding to lowest value
print("---")
print("The vector x =", opt_vector, "minimizes the objective function to a value of", int(min_value)) 

[Click here for solution](QUBO_Examples_TSP_Solutions.ipynb#task4)

***

### Task 5

Let's assume we have the following fully-connected, undirected graph of 5 nodes / cities and 10 edges, $G = (5,10)$. 

<img src="../images/tsp_2.png" width="250">

Determine the objective function for this graph including the constraints. You may assume suitable values for A and B. Furthermore, identify possible edges and corresponding costs. 

(Note: You don't need to solve and simplify the expression.) 

[Click here for solution](QUBO_Examples_TSP_Solutions.ipynb#task5)

***

### Task 6

Let's assume we have the following undirected graph of 5 nodes / cities and 8 edges, $G = (5,8)$. Note, its not fully connected. 

<img src="../images/tsp_4.png" width="250">

Determine the objective function for this graph including the constraints. You may assume suitable values for A and B. Furthermore, identify possible edges and corresponding costs. 

(Note: You don't need to solve and simplify the expression.)

[Click here for solution](QUBO_Examples_TSP_Solutions.ipynb#task6)

***

### References
***
1. Qiskit Tutorials. (n.d.). *Max-Cut and Traveling Salesman Problem.* Qiskit. [Link](https://qiskit.org/documentation/optimization/tutorials/06_examples_max_cut_and_tsp.html) 
2. Andrew Lucas. (2014). *Ising formulations of many NP problems.* Frontiers in Physics, Volume 2, Article 5. [Link](https://doi.org/10.3389/fphy.2014.00005)
3. Ozlem Salehi, & Adam Glos, & Jaroslaw Adam Miszczak. (2021). *Unconstrained Binary Models of the Travelling Salesman Problem Variants for Quantum Optimization.* [[arXiv Preprint]](https://arxiv.org/abs/2106.09056)